In [9]:
import cv2
import numpy as np

# Initializing the camera and taking the instance
cap = cv2.VideoCapture(0)

# Check if the camera is opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Load the pre-trained Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Status marking for the current state
sleep = 0
drowsy = 0
active = 0
status = ""
color = (0, 0, 0)

def compute(ptA, ptB):
    dist = np.linalg.norm(ptA - ptB)
    return dist

def blinked(a, b, c, d, e, f):
    up = compute(b, d) + compute(c, e)
    down = compute(a, f)
    ratio = up / (2.0 * down)

    # Checking if it is blinked
    if ratio > 0.35:  # Adjusted ratio for sleeping
        return 2
    elif 0.3 < ratio <= 0.35:  # Adjusted ratio for drowsy
        return 1
    else:
        return 0

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Check if the frame is valid
    if not ret:
        print("Error: Could not read frame.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform face detection using the Haar Cascade
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Assume eye regions based on face bounding box (you may need to fine-tune)
        left_eye = frame[y:y+h, x:x+w//2]
        right_eye = frame[y:y+h, x+w//2:x+w]

        # The numbers are actually the landmarks which will show eye
        left_blink = blinked(np.array((x, y)), np.array((x+w//2, y)),
                             np.array((x+w//4, y+h//4)), np.array((x+w//4, y+h*3//4)),
                             np.array((x+w//2, y+h*3//4)), np.array((x+w*3//4, y+h//4)))

        right_blink = blinked(np.array((x+w//2, y)), np.array((x+w, y)),
                              np.array((x+w*3//4, y+h//4)), np.array((x+w//2, y+h//4)),
                              np.array((x+w//2, y+h*3//4)), np.array((x+w*3//4, y+h*3//4)))

        # Now judge what to do for the eye blinks
        if left_blink == 0 or right_blink == 0:
            sleep += 1
            drowsy = 0
            active = 0
            if sleep > 6:
                status = "SLEEPING !!!"
                color = (255, 0, 0)
        elif 0.3 < left_blink <= 0.35 or 0.3 < right_blink <= 0.35:
            sleep = 0
            active = 0
            drowsy += 1
            if drowsy > 6:
                status = "Drowsy !"
                color = (0, 0, 255)
        else:
            drowsy = 0
            sleep = 0
            active += 1
            if active > 6:
                status = "Active :)"
                color = (0, 255, 0)

        # Draw eye regions (you may want to fine-tune or use a more sophisticated approach)
        cv2.rectangle(frame, (x, y), (x+w//2, y+h), (255, 255, 255), 2)
        cv2.rectangle(frame, (x+w//2, y), (x+w, y+h), (255, 255, 255), 2)

        cv2.putText(frame, status, (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()
